<a href="https://colab.research.google.com/github/pollyjuice74/REU-LDPC-Project/blob/main/LinearTranDiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/pollyjuice74/REU-LDPC-Project
!git clone https://github.com/pollyjuice74/DDECC
!wget https://raw.githubusercontent.com/NVlabs/sionna/main/sionna/utils/metrics.py -O metrics.py

Cloning into 'REU-LDPC-Project'...
remote: Enumerating objects: 701, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 701 (delta 185), reused 233 (delta 147), pack-reused 409
Receiving objects: 100% (701/701), 580.88 KiB | 4.54 MiB/s, done.
Resolving deltas: 100% (387/387), done.
Cloning into 'DDECC'...
remote: Enumerating objects: 613, done.
remote: Counting objects: 100% (345/345), done.
remote: Compressing objects: 100% (233/233), done.
remote: Total 613 (delta 191), reused 183 (delta 98), pack-reused 268
Receiving objects: 100% (613/613), 1.08 GiB | 30.89 MiB/s, done.
Resolving deltas: 100% (276/276), done.


In [2]:
import tensorflow as tf
import random
import numpy as np

import os
if os.path.exists('REU-LDPC-Project'):
  os.rename('REU-LDPC-Project', 'REU_LDPC_Project')
os.chdir('REU_LDPC_Project/adv_nn')

from attention import *
from channel import *
from decoder import *
from model import *
from dataset import *
from args import *

from metrics import BitErrorRate as ber
from metrics import compute_bler as bler
from DDECC.src.codes import EbN0_to_std

os.chdir('../../DDECC')

In [30]:

def train(model, train_loader, optimizer, epoch, LR):
    pass


from metrics import compute_bler as bler
def test(model, test_loader_list, EbNo_range_test, min_FER=100, max_cum_count=1e7, min_cum_count=1e5):
    printed = False
    ber_list, bler_list = [], []

    for ix, test_loader in enumerate(test_loader_list):
        for batch_ix, (m, c, z, r, _, _, magnitude, syndrome) in enumerate(test_loader):
            c_hat = model(r)
            ber_list.append(ber(c, c_hat))
            bler_list.append(bler(c, c_hat))
            print(f'Test EbN0={EbNo_range_test[ix]}, BER={ber_list[-1]}')
            print(f'Test EbN0={EbNo_range_test[ix]}, BLER={bler_list[-1]}')

            if not printed:
              print("c: ", c)
              print("c_hat: ", c_hat)
              printed = True
            break

    return { "ber": ber_list, "bler": bler_list }


def test_models():
    data = { "LTDM": dict() }

    for ix, tst_dataset in enumerate(test_ebnos_datasets):
        print(f"\nTesting on {list(dataset_types.keys())[ix].upper()}")
        print("Testing  Linear Transformer Diffusion Model...")
        data["LTDM"][ix] = test(dec, tst_dataset, EbNo_range_test, min_FER=50,max_cum_count=1e6,min_cum_count=1e4)
        return data


args = Args(model_type='dis') # args for decoder/discriminator
dataset = FEC_Dataset(args.code, args.sigma, zero_cw=True)
dec = Discriminator(args)

EbNo_range_train = range(2, 8)
EbNo_range_test = range(4, 8)
# Standard deviation for train/test
std_train = [EbN0_to_std(ii, args.k / args.n) for ii in EbNo_range_train]
std_test = [EbN0_to_std(ii, args.k / args.n) for ii in EbNo_range_test]

scheduler = tf.keras.optimizers.schedules.CosineDecay(initial_learning_rate=args.lr, decay_steps=args.epochs) # 1000 is size of trainloader
optimizer = tf.keras.optimizers.Adam(learning_rate=scheduler)

train_dataloader = FEC_Dataset(args.code, sigma=std_test, length=args.traindata_len) \
                              .batch(args.batch_size).shuffle(buffer_size=args.batch_size)
#                         z_cw   m 1s   1-cw     Should use zero codeword by default
dataset_types = {
              "awgn":    (False, False, False), # Binary bits sent and recieved with some awgn
              "flip_cw": (True, False, True),   # Zero codeword flipped to a all ones vector [1,1,...,1]
              "zero_cw": (True, False, False),  # Standard zero codeword used for training
              "ones_m":  (False, True, False),  # Makes the message all ones vector and passes it to generator matrix producing codeword and pcm
              }
test_ebnos_datasets = [ [FEC_Dataset(args.code, sigma=std_test, zero_cw=zero_cw, ones_m=ones_m, flip_cw=flip_cw)
                                 for ii in range(len(std_test))]
                                 for (zero_cw, ones_m, flip_cw) in dataset_types.values() ]


for epoch in range(1, args.epochs + 1):
    # print("Training Linear Transformer Diffusion Model...")
    # train(dec, train_dataloader, optimizer, epoch, LR=scheduler(tf.Variable(0, dtype=tf.float32)).numpy())

    # print comparison
    if epoch % 1 == 0:
        data = test_models()
    break # from for loop

data


Testing on AWGN
Testing  Linear Transformer Diffusion Model...


UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} NameError: name 'args' is not defined
Traceback (most recent call last):

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/ops/script_ops.py", line 270, in __call__
    ret = func(*args)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)

  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/from_generator_op.py", line 198, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/content/REU_LDPC_Project/adv_nn/dataset.py", line 39, in _generator
    z = tf.random.normal((args.code.n,), std_noise)

NameError: name 'args' is not defined


	 [[{{node PyFunc}}]] [Op:IteratorGetNext] name: 